**Hi every one**

**Q.** How much is readibility of text(excerpts)**?**

**A.** Data analysis and RoBerta model can be helpful!



**Thank you so much the notebooks:**

*  [CommonLit Readibility Prize Extensive EDA + Model](http://)
*  [CommonLit readability: RoBerta inference](http://)
*  [CommonLit Readability Prize - EDA](http://)

# **1. Importing modules and libraries**

In [ ]:
import numpy as np 
import pandas as pd 

import os
import re

import matplotlib as plt
import seaborn as sns
import itertools
import nltk 
import string
from wordcloud import WordCloud
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob,Word
from collections import Counter
import seaborn as sns
from tqdm.notebook import tqdm

import nltk
from nltk.corpus import stopwords
from collections import defaultdict

#visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly 
import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly.io as pio
from plotly.subplots import make_subplots
import plotly.express as px
from plotly.offline import iplot, init_notebook_mode
import cufflinks as cf
import plotly.figure_factory as ff 
from plotly.offline import iplot
from plotly import tools
colors = px.colors.qualitative.Prism
pio.templates.default = "plotly_white"

import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.offline import iplot
from wordcloud import WordCloud
from scipy.stats import probplot
from plotly.offline import iplot

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.metrics import mean_squared_error


import nltk
from nltk import tokenize
from nltk.corpus import wordnet, stopwords
from nltk.stem import WordNetLemmatizer

def cprint(string:str, end="\n"):
    """
    A little utility function for printing and stuff
    """
    _pprint(f"[black]{string}[/black]", end=end)
    
from transformers import *

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

import random

import torch
import torch.nn as nn

from torch.utils.data import DataLoader, Dataset

from tqdm import tqdm
    
    
# see our files:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        


# **2. Reading data**

In [ ]:
train_df = pd.read_csv('/kaggle/input/commonlitreadabilityprize/train.csv')
test_df = pd.read_csv('/kaggle/input/commonlitreadabilityprize/test.csv')
sample_submission = pd.read_csv('../input/commonlitreadabilityprize/sample_submission.csv')

# **3. Data analysis**

Let's see the data insight first!

In [ ]:
train_df.head()

In [ ]:
train_df.info()

Great, there aren't any null! 

In [ ]:
test_df.head()

In [ ]:
test_df.info()

In [ ]:
sample_submission.head()

**Target probability**

In [ ]:
def plot_target(target):
    
    print(f'{target}\n{"-" * len(target)}')
        
    print(f'Mean: {train_df[target].mean():.4}  -  Median: {train_df[target].median():.4}  -  Std: {train_df[target].std():.4}')
    print(f'Min: {train_df[target].min():.4}  -  25%: {train_df[target].quantile(0.25):.4}  -  50%: {train_df[target].quantile(0.5):.4}  -  75%: {train_df[target].quantile(0.75):.4}  -  Max: {train_df[target].max():.4}')
    print(f'Skew: {train_df[target].skew():.4}  -  Kurtosis: {train_df[target].kurtosis():.4}')
    missing_values_count = train_df[train_df[target].isnull()].shape[0]
    training_samples_count = train_df.shape[0]
    print(f'Missing Values: {missing_values_count}/{training_samples_count} ({missing_values_count * 100 / training_samples_count:.4}%)')

    fig, axes = plt.subplots(ncols=2, figsize=(24, 8), dpi=100)
    sns.kdeplot(train_df[target], label=target, fill=True, ax=axes[0])
    axes[0].axvline(train_df[target].mean(), label=f'{target} Mean', color='r', linewidth=2, linestyle='--')
    axes[0].axvline(train_df[target].median(), label=f'{target} Median', color='b', linewidth=2, linestyle='--')
    probplot(train_df[target], plot=axes[1])
    axes[0].legend(prop={'size': 15})
    
    for i in range(2):
        axes[i].tick_params(axis='x', labelsize=12)
        axes[i].tick_params(axis='y', labelsize=12)
        axes[i].set_xlabel('')
        axes[i].set_ylabel('')
    axes[0].set_title(f'{target} Distribution in Training Set', fontsize=15, pad=12)
    axes[1].set_title(f'{target} Probability Plot', fontsize=15, pad=12)

    plt.show()

plot_target('target')

**standard_error vs target**

In [ ]:
fig, ax = plt.subplots(figsize=(16, 6))
no_baseline = train_df['standard_error'] > 0
sns.scatterplot(x=train_df.loc[no_baseline, 'target'], y=train_df.loc[no_baseline, 'standard_error'], ax=ax)
ax.set_title(f'standard_error vs target', size=18, pad=15)
ax.set_xlabel('target', size=15, labelpad=12)
ax.set_ylabel('standard_error', size=15, labelpad=12)
ax.tick_params(axis='x', labelsize=12, pad=10)
ax.tick_params(axis='y', labelsize=12, pad=10)
plt.show()

**So, as a result,** around -1 we have more target and less error.

> **Target and sentence length**

In [ ]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

lemmatizer = WordNetLemmatizer()
def clean_text(text):
    text = re.sub('[^A-Za-z0-9]+', ' ', text.lower())
    words = nltk.word_tokenize(text)
    tagged = nltk.pos_tag(words)
    words = [lemmatizer.lemmatize(word, get_wordnet_pos(pos)) for word, pos in tagged]
    words = [word for word in words if word not in stopwords.words('english')]
    return words

def get_ngrams(words, n):
    return [tuple(words[i:i+n]) for i in range(len(words)-n+1)]

corpus = []
for text, target in train_df[['excerpt', 'target']].itertuples(index=False):
    sentences = []
    for sentence in tokenize.sent_tokenize(text):
        words = clean_text(sentence)
        unigrams = get_ngrams(words, n=1)
        bigrams = get_ngrams(words, n=2)
        trigrams = get_ngrams(words, n=3)
        sentences.append(words)
    corpus.append({
        'target' : target,
        'text' : text,
        'sentences' : sentences,
        'unigrams' : unigrams,
        'bigrams' : bigrams,
        'trigrams' : trigrams,
    })

corpus = sorted(corpus, key=lambda x: x['target'])

In [ ]:
top_lowest = corpus[:500]
lowest_target_sentence_lengths = [ \
    np.mean([len(sentence) for sentence in datapoint['sentences']]) \
    for datapoint in top_lowest \
]

top_highest = corpus[-500:]
highest_target_sentence_lengths = [ \
    np.mean([len(sentence) for sentence in datapoint['sentences']]) \
    for datapoint in top_highest \
]

top_lowest_mean = np.mean(lowest_target_sentence_lengths)
top_lowest_std = np.std(lowest_target_sentence_lengths)
top_highest_mean = np.mean(highest_target_sentence_lengths)
top_highest_std = np.std(highest_target_sentence_lengths)

fig, ax = plt.subplots(figsize=(8, 10), facecolor='#f6f5f5')
ax.errorbar(
    x=[0, 1],
    y=[top_lowest_mean, top_highest_mean],
    yerr=[top_lowest_std, top_highest_std],
    fmt='o'
)

ax.set_title('Average sentence length and Readability')
ax.set_ylabel('Sentence length')
ax.set_xticks([0, 1])
ax.set_xticklabels(['Top lowest readability', 'Top highest readability'])

plt.show()

> **So, as a result**: Texts with shorter sentence lengths are often easier to read.

And now, WordCloud to visuale the most common words in the all excerpts of train data!
Altough befor that it is needed to clean train data!

**Cleaning the data**

In [ ]:
def text_cleaning(excerpt):
    '''
    Converts all text to lower case, Removes special charecters, emojis and multiple spaces
    text - Sentence that needs to be cleaned
    '''
    excerpt = ''.join([k for k in text if k not in string.punctuation])
    excerpt = re.sub('[^A-Za-z0-9]+', ' ', str(text).lower()).strip()
#     text = re.sub("/'+/g", ' ', text)
    
    return text

In [ ]:
%%time
tqdm.pandas()
train_df['excerpt'] = train_df['excerpt'].progress_apply(text_cleaning)

In [ ]:
train_df['temp_list'] = train_df['excerpt'].apply(lambda x:str(x).split())
top = Counter([item for sublist in train_df['temp_list'] for item in sublist])
temp = pd.DataFrame(top.most_common(20))
temp.columns = ['Common_words','count']
temp.style.background_gradient(cmap='Blues')

In [ ]:
from wordcloud import WordCloud, STOPWORDS
stopwords = set(STOPWORDS)

def show_wordcloud(data, title = None):
    wordcloud = WordCloud(
        background_color='black',
        stopwords=stopwords,
        max_words=200,
        max_font_size=40, 
        scale=3,
        random_state=1 # chosen at random by flipping a coin; it was heads
).generate(str(data))

    fig = plt.figure(1, figsize=(15, 15))
    plt.axis('off')
    if title: 
        fig.suptitle(title, fontsize=20)
        fig.subplots_adjust(top=2.3)

    plt.imshow(wordcloud)
    plt.show()

show_wordcloud(train_df['excerpt'])

# 4. Modelling

**RoBerta model**

a. Training the model

In [ ]:
import numpy as np 
import pandas as pd 

import os
from transformers import *

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

import random

import torch
import torch.nn as nn

from torch.utils.data import DataLoader, Dataset

from tqdm import tqdm

In [ ]:
def set_seed(seed = 0):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random_state = np.random.RandomState(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    return random_state

seed = 82
random_state = set_seed(seed)

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

In [ ]:
train_df = pd.read_csv('../input/commonlitreadabilityprize/train.csv')

In [ ]:
train_df['excerpt_len'] = train_df.excerpt.apply(lambda x: len(x.split()))

In [ ]:
print(train_df.excerpt_len.max())

In [ ]:
train_df['fold'] = -1
gkf = KFold(n_splits=5)
for fold, (train, val) in enumerate(gkf.split(train_df.excerpt, train_df.target)):
    train_df.loc[val,'fold']=fold

fold = 0
validation_df = train_df[train_df.fold==0].reset_index(drop=True)
train_df = train_df[train_df.fold!=0].reset_index(drop=True)

In [ ]:
print(train_df.target.mean(), validation_df.target.mean())

In [ ]:
class Data(Dataset):
    def __init__(self, data):
        super().__init__()
        self.data = data

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):       
        excerpt = self.data.excerpt[idx]
        target = self.data.target[idx]
        return excerpt, target

In [ ]:
train_data = Data(data = train_df) 
train_loader = DataLoader(dataset = train_data, shuffle=True, batch_size = 8)

val_data = Data(data = validation_df) 
val_loader = DataLoader(dataset = val_data, shuffle=False, batch_size = 64)

In [ ]:
class ReadabilityModel(PreTrainedModel): 
    def __init__(self, conf):
        super(ReadabilityModel, self).__init__(conf) 
        self.roberta = RobertaModel.from_pretrained(model_name, config=conf)
        self.drop_out = nn.Dropout(0.1)
        self.l1 = nn.Linear(768 * 1, 1)
        torch.nn.init.normal_(self.l1.weight, std=0.02)
    
    def forward(self, ids, mask):
        out = self.roberta(
            input_ids=ids,
            attention_mask=mask
        )
        out = out['hidden_states']
        out = out[-1]
        out = self.drop_out(out)
        out = torch.mean(out, 1, True)
        
        preds = self.l1(out)

        preds = preds.squeeze(-1).squeeze(-1)

        return preds

In [ ]:
model_name = 'roberta-base'
tokenizer = RobertaTokenizerFast.from_pretrained(model_name)

model_config = RobertaConfig.from_pretrained(model_name)
model_config.output_hidden_states = True

model = ReadabilityModel(model_config)
model = model.to(device)

optimizer = AdamW(model.parameters(), lr=1e-5, weight_decay=0.01)
scheduler = get_constant_schedule_with_warmup(optimizer, 100)

loss_fct = nn.MSELoss()

epochs = 3

In [ ]:
for epoch in range(epochs):
    model.train()
    for i, (excerpts, targets) in enumerate(tqdm(train_loader)):
        optimizer.zero_grad()
        batch = tokenizer(list(excerpts), truncation=True, padding=True, return_tensors='pt', add_special_tokens=True)
        input_ids = batch['input_ids']
        input_ids = input_ids.to(device, dtype=torch.long)
        attention_mask = batch['attention_mask']
        attention_mask = attention_mask.to(device, dtype=torch.long)
            
        targets=torch.tensor(targets).to(device, dtype=torch.float)
 
        preds = model(input_ids, attention_mask)       
        
        loss = torch.sqrt(loss_fct(preds, targets))
        
        loss.backward()
        optimizer.step()
        scheduler.step()
        loss = loss.item()
        
        if i==0:
            loss_train = loss
        else:
            loss_train = loss_train + loss  
    loss_train = loss_train/(i+1)
    
    model.eval()
    with torch.no_grad():
        for i, (excerpts, targets) in enumerate(tqdm(val_loader)):
            optimizer.zero_grad()
            batch = tokenizer(list(excerpts), truncation=True, padding=True, return_tensors='pt', add_special_tokens=True)
            input_ids = batch['input_ids']
            input_ids = input_ids.to(device, dtype=torch.long)
            attention_mask = batch['attention_mask']
            attention_mask = attention_mask.to(device, dtype=torch.long)
                
            targets=torch.tensor(targets).to(device, dtype=torch.float)
     
            preds = model(input_ids, attention_mask)       
            
            loss = torch.sqrt(loss_fct(preds, targets))
            loss = loss.item()
            
            preds = preds.cpu().detach().numpy()
            targets = targets.cpu().detach().numpy()
            if i==0:
                loss_val = loss
                preds_val = preds
                targets_val = targets
            else:
                loss_val = loss_val + loss  
                preds_val = np.concatenate((preds_val,preds), axis=None)
                targets_val = np.concatenate((targets_val,targets), axis=None)
                
        loss_val = loss_val / (i+1)
        rms_val = mean_squared_error(targets_val, preds_val, squared=False)
        print('Epoch: {} - Loss: {:.6f} - Loss val: {:.6f} - RMSE: {:.3f}'.format(
            epoch + 1, loss_train, loss_val, rms_val))

In [ ]:
torch.save(model.state_dict(), 'roberta_baseline.bin')

b. Inferencing the  model

In [ ]:
test_df = pd.read_csv('../input/commonlitreadabilityprize/test.csv')

In [ ]:
test_df

In [ ]:
class Data(Dataset):
    def __init__(self, data):
        super().__init__()
        self.data = data

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):       
        excerpt = self.data.excerpt[idx]
        return excerpt

In [ ]:
test_data = Data(data = test_df) 
test_loader = DataLoader(dataset = test_data, shuffle=False, batch_size = 64)

In [ ]:
class ReadabilityModel(PreTrainedModel): 
    def __init__(self, conf):
        super(ReadabilityModel, self).__init__(conf) 
        self.roberta = RobertaModel(config=conf)
        self.drop_out = nn.Dropout(0.1)
        self.l1 = nn.Linear(768 * 1, 1)
        torch.nn.init.normal_(self.l1.weight, std=0.02)
    
    def forward(self, ids, mask):
        out = self.roberta(
            input_ids=ids,
            attention_mask=mask
        )
        out = out['hidden_states']
        out = out[-1]
        out = self.drop_out(out)
        out = torch.mean(out, 1, True)
        
        preds = self.l1(out)

        preds = preds.squeeze(-1).squeeze(-1)

        return preds

In [ ]:
tokenizer = RobertaTokenizerFast.from_pretrained('../input/robertabase', model_max_length=514) 

model_config = RobertaConfig()
model_config.output_hidden_states = True
model_config.max_position_embeddings=514
model_config.vocab_size = 50265
model_config.type_vocab_size = 1

model = ReadabilityModel(model_config)
if torch.cuda.is_available():
    model.load_state_dict(torch.load("../input/commonlit-readability-roberta-simple-baseline/roberta_baseline.bin"))
else: 
    model.load_state_dict(torch.load("../input/commonlit-readability-roberta-simple-baseline/roberta_baseline.bin", map_location=torch.device('cpu')))
model = model.to(device)

In [ ]:
model.eval()
with torch.no_grad():
    for i, excerpts in enumerate(tqdm(test_loader)):
        batch = tokenizer(list(excerpts), truncation=True, padding=True, return_tensors='pt', add_special_tokens=False)
        input_ids = batch['input_ids']
        input_ids = input_ids.to(device, dtype=torch.long)
        attention_mask = batch['attention_mask']
        attention_mask = attention_mask.to(device, dtype=torch.long)
            
        preds = model(input_ids, attention_mask)       
        preds = preds.cpu().detach().numpy()

        if i==0:
            preds_test = preds
        else:
            preds_test = np.concatenate((preds_test,preds), axis=None)

In [ ]:
submission = pd.DataFrame({'id': test_df.id, 'target': preds_test})

In [ ]:
submission.to_csv('/kaggle/working/submission.csv', index=False)

In [ ]:
submission